In [37]:
data class WorkItem(
    val v: Vertex,
    val steps: Int,
    val used: Edge?,
    val prev: WorkItem?
)

data class Vertex(var label: String?) {
    val outgoing: MutableList<Edge> = mutableListOf<Edge>()

    fun addNeighbour(edge: Edge) {
        outgoing.add(edge)
    }
}

data class Edge(val one: Vertex, val two: Vertex, var counter: Int = 0) {

    fun other(v: Vertex): Vertex {
        return if (v === one) two else one
    }
}

fun flood(
    start: Vertex,
    disconnected: MutableSet<Edge>
): Int {
    val seen = mutableSetOf<Vertex>()
    val todo = ArrayDeque<Vertex>()
    todo.add(start)
    while (!todo.isEmpty()) {
        val cur: Vertex = todo.removeFirst()
        if (seen.contains(cur)) {
            continue
        }
        seen.add(cur)
        for (e in cur.outgoing) {
            if (disconnected.contains(e)) {
                continue
            }
            todo.add(e.other(cur))
        }
    }
    return seen.size
}

fun dijkstra(start: Vertex, end: Vertex) {
    val todo = PriorityQueue<WorkItem> { o1, o2 -> o1.steps.compareTo(o2.steps) }
    val shortest = mutableMapOf<Vertex, Int>()
    todo.add(WorkItem(start, 0, null, null))
    while (!todo.isEmpty()) {
        val wi: WorkItem = todo.remove()
        val way = shortest.getOrPut(wi.v) { Integer.MAX_VALUE }

        if (way <= wi.steps) {
            continue
        }
        if (end === wi.v) {
            var cur: WorkItem? = wi
            while (cur?.used != null) {
                (cur.used as Edge).counter++
                cur = cur.prev
            }
            return
        }
        shortest[wi.v] = wi.steps
        for (e in wi.v.outgoing) {
            val cand: Vertex = e.other(wi.v)
            val cur = shortest.getOrPut(cand) { Integer.MAX_VALUE }
            if (cur > wi.steps + 1) {
                todo.add(WorkItem(cand, wi.steps + 1, e, wi))
            }
        }
    }
}

fun dfs(start: Vertex, n: Int) {
    val visited = mutableSetOf<Vertex>()
    val todo = ArrayDeque<Vertex>()
    val todo2 = ArrayDeque<Edge>()
    for (e in start.outgoing) {
        todo2.addFirst(e)
        todo.addFirst(e.other(start))
    }
    while (visited.size < n) {
        val vertex: Vertex = todo.removeLast()
        val edge: Edge = todo2.removeLast()
        if (!visited.contains(vertex)) {
            visited.add(vertex)
            edge.counter++
            for (e in vertex.outgoing) {
                todo2.addFirst(e)
                todo.addFirst(e.other(vertex))
            }
        }
    }
}

val vertices = mutableMapOf<String, Vertex>()
val edges = mutableListOf<Edge>()

File("input.txt").forEachLine { s ->
    val parts = s.split(": ".toRegex()).dropLastWhile { it.isEmpty() }.toTypedArray()
    val nv = vertices.computeIfAbsent(parts[0]) { Vertex(it) }
    val neighbours = parts[1].split(" ".toRegex()).dropLastWhile { it.isEmpty() }
        .toTypedArray()
    for (nb in neighbours) {
        val nv2 = vertices.computeIfAbsent(nb) { Vertex(it) }

        val edge = Edge(nv, nv2)
        edges.add(edge)
        nv.addNeighbour(edge)
        nv2.addNeighbour(edge)
    }
}

val tries = 3000
val vertexList = vertices.values.toMutableList()
val r = Random.Default
for (i in 0 until tries) {
    val start: Vertex = vertexList[r.nextInt(vertices.size)]
    val end: Vertex = vertexList[r.nextInt(vertices.size)]
    dijkstra(start, end)
}

edges.sortWith(Comparator<Edge> { o1: Edge, o2: Edge -> o2.counter - o1.counter })
val disconnected = edges.subList(0, 3).toMutableSet()
val size1 = flood(edges.first().one, disconnected)
val size2 = flood(edges.first().two, disconnected)
size1 * size2

596376